This is a notebook with various methods for processing the Nino 3.4, PDO and AMO time series found on NOAA websites into useable data

In [66]:
import os
import datetime as dt
import matplotlib.pyplot as plt

In [79]:
filepath = r'C:\Users\bpara\Geo data\climate oscillations'
amo_s_psl_p = os.path.join(filepath, "amo smoothed.txt")
amo_r_psl_p = os.path.join(filepath, "amo raw.txt")
nino_r_psl_p = os.path.join(filepath, "nino raw data.txt")
pdo_r_csv_p = os.path.join(filepath, "pdo raw.csv")

In [9]:
os.listdir(filepath)

['amo smoothed.txt', 'nino raw data.txt', 'pdo raw.csv']

In [81]:
#This is a function that imports standard psl format data when it is copy-pasted into a .txt file, for points with no data, a value of None is inserted at that index

def import_psl_format(filepath, start_year, end_year, datakey):
    raw_text = open(filepath, "r")
    final_data = dict()
    final_data["time"] = []
    final_data[datakey] = []
    
    for x in raw_text:
        year_data = x.split()[1:]
        month = 1
        current_year = int(x.split()[0])
        if current_year < start_year or current_year > end_year: 
            continue #This is a bit hack-y, but it's a fast way of doing this
        for val in year_data:
            final_data["time"].append(dt.datetime(current_year, month, 1))
            if float(val) != -99.990:
                final_data[datakey].append(float(val))
            else:
                final_data[datakey].append(None)
            month += 1
    
    
    raw_text.close()
    
    return final_data

In [82]:
amo_s = import_psl_format(amo_s_psl_p, 1980, 2019, "amo")
nino_r = import_psl_format(nino_r_psl_p, 1980, 2019, "nino")
amo_r = import_psl_format(amo_r_psl_p, 1979, 2020, "amo")

In [109]:
#This function takes your dataset and returns a smoothed dataset generated by averaging each month with the n previous months and n upcoming months (so like a 3-month rolling
#average would be made with n=1)

def n_month_average(data, datakey, months):
    final_data = dict()
    final_data["time"] = []
    final_data[datakey] = []
    
    for x in range(len(data["time"])):
        if x - months > 0 and x + months < len(data["time"]) - 1:
            average_vals = data[datakey][x - months : x + months]
            total = 0
            for val in average_vals:
                if val == None:
                    total = None
                    break
                total += val
            if total == None:
                final_data["time"].append(data["time"][x])
                final_data[datakey].append(None)
                continue
            final_data["time"].append(data["time"][x])
            final_data[datakey].append(total / (2 * months + 1))
            
    return final_data

In [111]:
my_smooth = n_month_average(amo_r, "amo", 60)

In [119]:
amo_s["time"][49]

datetime.datetime(1984, 2, 1, 0, 0)

In [83]:
#This is a function that takes in the raw nino data, and for each time interval, it finds the number of nino events in a 8 year window (average time between Nino events is 
#about 4 years, so with this you'll usually hit around 2). It does this by looking at the 8 years after a certain date and noting the number of periods of consequitive months with 
#greater than 0.5 temp anomaly (if you're looking for nino events) or less than -0.5 temp anomaly (if you're looking for nina events) 

def smooth_nino_data(nino_data, month_window, nino_or_nina):
        final_data = dict()
        final_data["time"] = nino_data["time"]
        final_data["nino"] = []
        
        for month in range(len(final_data["time"])):
            has_counted = False
            
        
        